[筆記來源](https://colab.research.google.com/drive/15Cyy2H7nT40sGR7TBN5wBvgTd57mVKay#forceEdit=true&sandboxMode=true&scrollTo=hG9gxhAqVTBT)

> Regression: Used to predict a numeric value  
> Classification: Used to <font color='blue'>seperate data points into classes of different labels.</font>

#### Project: 用一個 TensorFlow estimator 來 classify flowers.

#### Dataset
Dataset 共有三種不同的種類:
- Setosa
- Versicolor
- Virginica

每種花都有下列資訊:
- sepal length 花萼長度
- sepal width 花萼寬度
- petal length 花瓣長度
- petal width 花瓣寬度

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

In [2]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']

SPECIES = ['Setosa', 'Versicolor', 'Virginica']

# 創建可能會用到的資訊。

#### 每次做 ML 的 dataset 來源都不一樣，所以重點是架構，而非程式碼。
#### 這次花的 dataset 來自 keras，keras 可以想成是 TensorFlow 的一個 sub module
#### 用 <font color='blue'>tf.keras.utils.get_file()</font> 將資料讀取
#### 之後再將其<font color='blue'>變成 DataFrame 模式。</font>

In [3]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)

In [4]:
type(train)

pandas.core.frame.DataFrame

In [5]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


##### 注意到最後一個 column，也就是我們的 label。已經是 numerical data 了。

In [6]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [7]:
train # 成功將 Species column 去除了

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3
...,...,...,...,...
115,5.5,2.6,4.4,1.2
116,5.7,3.0,4.2,1.2
117,4.4,2.9,1.4,0.2
118,4.8,3.0,1.4,0.1


In [8]:
train_y # label 在這裡

0      2
1      1
2      2
3      0
4      0
      ..
115    1
116    1
117    0
118    0
119    1
Name: Species, Length: 120, dtype: int64

#### 設定 feature column:
##### 這裡較 linear regression 的例子簡單，因為所有 features 已經是 numerical 了。

[df.keys()](https://www.geeksforgeeks.org/python-pandas-dataframe-keys/)

> Pandas dataframe.keys() function returns the ‘info axis’ for the pandas object.  
> If the pandas object is series: it returns index.   
> <font color='green'>If the pandas object is dataframe: it returns columns. </font>  
> If the pandas object is panel: it returns major_axis.

In [9]:
my_feature_columns = []

for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


#### 跟 Linear Regression 的寫法比較: (其實概念是一樣的)
```python
for feature_name in NUMERIC_COLUMNS:  
    feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))
```

#### 現在要設定 input function:
##### 但這個 input function 看上去比 linear regression 的 input function 簡單。
##### 注意到這次我們沒有給 epoch
##### <font color='red'> 初學階段不用因為寫不出 input function 而沮喪，看多了，自然就會寫了。</font>

In [10]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

#### 如果要做 classification，有很多種 models/estimaters 可以選，常見的兩個如下:
- DNNClassifier (Deep Neural Network)
- LinearClassifier
##### 建議用 DNNClassifier，因為 LinearClassifier 不一定找的到線性關係。

In [11]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each.
classifier = tf.estimator.DNNClassifier( # tf.estimater內存許多不同的 models。
    feature_columns=my_feature_columns,
    hidden_units=[30, 10], # 給定兩層 hidden layers，依序有 30 跟 10 個 nodes。
    n_classes=3) # n_classes 將指定電腦選擇 3 種 classes(因為我們知道總共有三種品種的花)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\User\\AppData\\Local\\Temp\\tmpgyumtxzl', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000028F25B6D630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [12]:
type(classifier)

tensorflow_estimator.python.estimator.canned.dnn.DNNClassifier

#### 有了 classifier，我們就可以 train 了!  
##### 注意到我們用了 <font color='red'> lambda </font> [參考資料](https://medium.com/citycoddee/python%E9%80%B2%E9%9A%8E%E6%8A%80%E5%B7%A7-4-lambda-function-%E8%88%87-closure-%E4%B9%8B%E8%AC%8E-7a385a35e1d8)
##### lambda 基本上跟 function 裡面再有 function 是一樣的概念(在 linear regression 時 input function 即為此)
---
##### 用 code 舉例:
```python
x = lambda: print('hi')
x() # 印出 hi
```
##### 意思是我們呼叫 x，而 x 會再執行一個 function(function 的內容是冒號後面的 code)，最終印出 hi。

In [13]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=5000) # steps(這次沒有 epoch) 是 500。

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\User\AppData\Local\Temp\tmpgyumtxzl\model.ckpt.
INFO:tensorflow:loss = 488.8822, step = 1
INFO:tensorflow:global_step/sec: 388.224
INFO:tensorflow:loss = 97.15818, step = 101 (0.259 sec)
INFO:tensorflow:global_step

In [14]:
x = lambda x: len(x)
print(x('x'))
type(x)

1


function

#### train 完後可以讓他考試了:

In [15]:
eval_result = classifier.evaluate(input_fn=lambda: input_fn(test, test_y, training=False)) # training 要記得設為 False

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-09-18T11:30:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmpgyumtxzl\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-09-18-11:30:33
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.96666664, average_loss = 0.08001992, global_step = 5000, loss = 2.4005976
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: C:\Users\User\AppData\Local\Temp\tmpgyumtxzl\model.ckpt-5000


In [16]:
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.967



#### 結果看起來不錯，現在設計一個方法，讓 user input features(sepal length, sepal width, petal length, petal width) 並 predict 可能是哪一種花。

In [17]:
def input_fn(features, batch_size=256): # 注意到這次的參數，只有 fectures(沒有 train, train_y 等)
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']

predict = {} # 傳入 input function 的 features 是一個字典

# 接著輸入者必須輸入正確的型態(數字型態)才可以丟入 predict
print("輸入數字資料: ")

for feature in features:
    valid = True
    while valid: 
        val = input(feature + ": ") # input(設計使用者看到的介面) 
        if not val.isdigit():
            valid = False
            '''
                這裡作者的意思是: 當 val 有不是數字的 input(小數點也算)，就讓 valid = False 進而結束迴圈。
                因為在這個 case，花瓣花萼的長寬應該都小於 10。
            '''
    predict[feature] = [float(val)] # 字典的 key 是目前 feature，value 則一定要用 list 包起來。
    
print(predict)

輸入數字資料: 
SepalLength: 2.3
SepalWidth: 3.5
PetalLength: 4.5
PetalWidth: 8.1
{'SepalLength': [2.3], 'SepalWidth': [3.5], 'PetalLength': [4.5], 'PetalWidth': [8.1]}


In [22]:
predictions = classifier.predict(input_fn=lambda: input_fn(predict)) 
# 第一個 input_fn 是參數，第二個 input function 是自己設的
print(predictions) 

<generator object Estimator.predict at 0x0000028F25AD97C8>


#### predictions 是一個 generator[作者有提到 generator 物件適合用 for 迴圈取出裡面的資料]
[Generator 生成器](https://www.itread01.com/content/1543162922.html)

In [19]:
for pred_dict in predictions:
    print(pred_dict)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmpgyumtxzl\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ -7.769993, -11.040387,  28.50204 ], dtype=float32), 'probabilities': array([1.7670788e-16, 6.7133945e-18, 1.0000000e+00], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}



- class_ids 是一個 array，2 代表電腦根據我們輸入的資料，認為他是 SPECIES = ['Setosa', 'Versicolor', 'Virginica'] 中的第 2 編號(也就是 Virginica)
- probabilities 是一個 array，3 個數字代表電腦認為是各個物種的機率(注意到最後一個數字最大，因此電腦認為是 Virginica)

In [23]:
for pred_dict in predictions:
    print(pred_dict)
    class_id = pred_dict['class_ids'][0] 
    probability = pred_dict['probabilities'][class_id]

    print(class_id)
    print('Prediction is "{}" ({:.1f}%)'.format(SPECIES[class_id], 100 * probability))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\User\AppData\Local\Temp\tmpgyumtxzl\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ -7.769993, -11.040387,  28.50204 ], dtype=float32), 'probabilities': array([1.7670788e-16, 6.7133945e-18, 1.0000000e+00], dtype=float32), 'class_ids': array([2], dtype=int64), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2]), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
2
Prediction is "Virginica" (100.0%)
